# **The Bitcoin Wallet**

## **BIP 39**

### Create a random integer that can serve as a securely generated seed for a wallet

In [92]:
import secrets

def generate_secure_seed(bits=128):
    # Generate a random seed (entropy)
    return secrets.randbits(bits)

# Generate the seed
seed = generate_secure_seed()
print(f"Generated seed: {seed}")

Generated seed: 275430278602789470365765935520496708628


### Convert this seed to binary and split it into 11-bit chunks

In [93]:
import hashlib

def calculate_checksum(seed, bits=128):
    # Convert seed to bytes
    seed_bytes = seed.to_bytes(bits // 8, byteorder='big')
    
    # Calculate the SHA-256 hash of the seed
    hash_digest = hashlib.sha256(seed_bytes).hexdigest()
    
    # Convert the hash to binary and take the first few bits (bits / 32)
    checksum_length = bits // 32
    checksum = bin(int(hash_digest, 16))[2:].zfill(256)[:checksum_length]
    
    return checksum

def seed_to_binary_with_checksum(seed, bits=128):
    # Convert seed to binary string
    binary_seed = bin(seed)[2:].zfill(bits)
    
    # Calculate and append the checksum
    checksum = calculate_checksum(seed, bits)
    return binary_seed + checksum

def split_into_chunks(binary_seed, chunk_size=11):
    # Split binary seed into chunks of 11 bits
    return [binary_seed[i:i+chunk_size] for i in range(0, len(binary_seed), chunk_size)]

# Convert the seed to binary with checksum and split into 11-bit chunks
binary_seed_with_checksum = seed_to_binary_with_checksum(seed)
binary_chunks = split_into_chunks(binary_seed_with_checksum)

print(f"Seed in binary split into 11-bit chunks: {binary_chunks}")

Seed in binary split into 11-bit chunks: ['11001111001', '10101111100', '01001011100', '11101100100', '00100111011', '10010111110', '00101011110', '01011111010', '10110000111', '00100011000', '11011011000', '00101000001']


### Assign each chunk a word from the BIP 39 list and display the seed in mnemonic form

In [94]:
# Load the BIP 39 word list from a local file or URL
def load_bip39_wordlist(filepath="english.txt"):
    with open(filepath, 'r') as file:
        wordlist = file.read().splitlines()
    return wordlist

def binary_to_mnemonic(binary_chunks, wordlist):
    # Convert each 11-bit chunk into an integer
    indices = [int(chunk, 2) for chunk in binary_chunks]
    
    # Map each integer to a word from the BIP 39 word list
    mnemonic = [wordlist[index] for index in indices]
    
    return ' '.join(mnemonic)

# Load the BIP 39 word list
wordlist = load_bip39_wordlist()

# Use the seed that was generated and converted into binary_chunks in previous steps
mnemonic_phrase = binary_to_mnemonic(binary_chunks, wordlist)

# Print the mnemonic seed phrase
print(f"Mnemonic seed phrase: {mnemonic_phrase}")

Mnemonic seed phrase: song question entire uncle cherry oak cloth gap rail cart swallow choice


### Allow the import of a mnemonic seed

In [95]:
def mnemonic_to_seed(mnemonic_phrase, wordlist):
    # Split the mnemonic phrase into individual words
    words = mnemonic_phrase.split()
    
    # Check if all words exist in the BIP 39 word list
    if not all(word in wordlist for word in words):
        raise ValueError("One or more words in the mnemonic phrase are invalid.")
    
    # Convert each word into its index in the BIP 39 word list
    indices = [wordlist.index(word) for word in words]
    
    # Convert each index into a binary string of 11 bits
    binary_seed = ''.join([bin(index)[2:].zfill(11) for index in indices])
    
    # The original seed should be the first bits - the last few bits are the checksum
    checksum_length = len(words) * 11 // 33
    seed_bits = binary_seed[:-checksum_length]
    
    # Convert the binary seed back to an integer
    seed = int(seed_bits, 2)
    
    return seed

imported_seed = mnemonic_to_seed(mnemonic_phrase, wordlist)

# Confirm if the generated seed matches the imported seed
if seed == imported_seed:
    print("The generated seed and the imported seed are identical.")
else:
    print("The generated seed and the imported seed are different.")

print(f"Seed derived from the imported mnemonic: {imported_seed}")


The generated seed and the imported seed are identical.
Seed derived from the imported mnemonic: 275430278602789470365765935520496708628


## **BIP 32**

### Extract the master private key and chain code

In [99]:
import hmac
import hashlib

def hmac_sha512(key, data):
    return hmac.new(key, data, hashlib.sha512).digest()

def derive_master_keys(seed):
    hmac_key = b"Bitcoin seed"
    I = hmac_sha512(hmac_key, seed)
    
    # Split the result into master private key and chain code
    master_private_key = I[:32]
    chain_code = I[32:]
    
    return master_private_key, chain_code

# Convert the seed (integer) to bytes
hex_seed = seed.to_bytes((seed.bit_length() + 7) // 8, byteorder='big')

# Derive master private key and chain code
master_private_key, chain_code = derive_master_keys(hex_seed)

print(f"Master Private Key: {master_private_key.hex()}")
print(f"Chain Code: {chain_code.hex()}")

Master Private Key: 4befc1f9d6475469cf940a9b955962089d9588efef95219f5f30eac460939adf
Chain Code: 09fa3eb83ed59c3e7f297e959327e1dafd24a78d52933782da92b3c94baba3a3


### Extract the master public key

In [112]:
from ecdsa import SigningKey, SECP256k1

def get_master_public_key(master_private_key):
    # Generate signing key from the master private key using SECP256k1
    sk = SigningKey.from_string(master_private_key, curve=SECP256k1)
    
    # Get the verifying key (public key)
    vk = sk.verifying_key
    
    return vk.to_string("compressed")  # Return the compressed public key

master_public_key = get_master_public_key(master_private_key)
print(f"Master Public Key: {master_public_key.hex()}")

Master Public Key: 03f80da6e35d4e1602e310ab69b62158b554f9838290bc2ced5d2beebb421e0bbc


### Generate a child key

In [113]:
import hmac
import hashlib

def derive_child_key(parent_private_key, parent_chain_code, index):
    # Hardened derivation if index >= 0x80000000
    if index >= 0x80000000:
        data = b'\x00' + parent_private_key + index.to_bytes(4, 'big')
    else:
        public_key = get_master_public_key(parent_private_key)
        data = public_key + index.to_bytes(4, 'big')
    
    I = hmac.new(parent_chain_code, data, hashlib.sha512).digest()
    child_private_key = (int.from_bytes(I[:32], 'big') + int.from_bytes(parent_private_key, 'big')) % SECP256k1.order
    child_chain_code = I[32:]
    
    return child_private_key.to_bytes(32, 'big'), child_chain_code

# Example usage (using index 0 for simplicity)
index = 0
child_private_key, child_chain_code = derive_child_key(master_private_key, chain_code, index)

print(f"Child Private Key: {child_private_key.hex()}")
print(f"Child Chain Code: {child_chain_code.hex()}")

Child Private Key: 01007004c90f98391c863128f7837638d58198362d667a00cf6de4e9b8392117
Child Chain Code: 6e450f8aaf6f489234c158d46ae63613c165dd808820e07c9e58cd7b9c2074f4


### Generate a child key at index N

In [114]:
index_N = 5  # Replace with any desired index
child_private_key_N, child_chain_code_N = derive_child_key(master_private_key, chain_code, index_N)

print(f"Child Private Key at index {index_N}: {child_private_key_N.hex()}")
print(f"Child Chain Code at index {index_N}: {child_chain_code_N.hex()}")

Child Private Key at index 5: 6df0522591a67bf471e2ce903bd8e81f3d255429f902575fc57874943ec0e24c
Child Chain Code at index 5: e2021689a55ced774858fbc2022714435d98b889e029665f14b79b4fbfcb6d7f


### Generate a child key at index N with derivation level M

In [115]:
def derive_key_at_level(parent_private_key, parent_chain_code, index_N, level_M):
    current_private_key = parent_private_key
    current_chain_code = parent_chain_code
    
    for _ in range(level_M):
        current_private_key, current_chain_code = derive_child_key(current_private_key, current_chain_code, index_N)
    
    return current_private_key, current_chain_code

index_N = 5  # Replace with any index
level_M = 3  # Replace with the desired derivation level
child_private_key_M, child_chain_code_M = derive_key_at_level(master_private_key, chain_code, index_N, level_M)

print(f"Child Private Key at index {index_N} and level {level_M}: {child_private_key_M.hex()}")
print(f"Child Chain Code at index {index_N} and level {level_M}: {child_chain_code_M.hex()}")

Child Private Key at index 5 and level 3: 029f74bb5c13bcf6fca55636fd3cf3270754192e5808a86c5cde8ee6605b48d9
Child Chain Code at index 5 and level 3: 961fe300d3f741f08e175100c758b2733603ce1cc888b8dd66531071bf884f11
